Toronto–NYC Fire Incident Gold Data Harmonization & EDA

This notebook reviews and harmonizes Toronto and New York City fire incident **Gold-layer outputs** into a unified, model-ready dataset. It begins with exploratory data analysis (EDA), including schema validation, missingness checks, timestamp consistency, and distributional comparisons across both cities.

The notebook then applies harmonization logic to standardize incident identifiers, timestamps, boroughs, and incident types. The final Gold dataset is incident-level (first arrival) and enriched with response-time labels, survival analysis fields, and temporal features to support comparative EDA, predictive modeling, and survival analysis.


## 1. Import and Load Tables

In [0]:
from pyspark.sql import functions as F
from pyspark.sql import Window
from pyspark.sql import DataFrame

# --- DATA TABLE DIRECTORIES ---
TORONTO_GOLD_TABLE = "workspace.capstone_project.tfs_incidents_gold"   
NYC_GOLD_TABLE     = "workspace.capstone_project.nyc_fire_incidents_gold"



### 2.1 Load Tables

In [0]:
toronto_gold = spark.table(TORONTO_GOLD_TABLE)
nyc_gold     = spark.table(NYC_GOLD_TABLE)

print("Toronto Gold count:", toronto_gold.count())
print("NYC Gold count:", nyc_gold.count())

display(toronto_gold.limit(5))
display(nyc_gold.limit(5))

toronto_gold.printSchema()
nyc_gold.printSchema()
